<a href="https://colab.research.google.com/github/munroalex/eve_related/blob/master/eveskillboard_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setting up the API calls required for eveskillboard.

In [1]:
# Imports required
import numpy as np
import json
import pandas as pd
import base64
import requests
import time
from requests.exceptions import HTTPError

In [56]:
# Define the scopes required here, ensure they are matching with the ones on your CCP developer application
scopes = ["publicData", 
          "esi-location.read_location.v1", 
          "esi-location.read_ship_type.v1", 
          "esi-skills.read_skills.v1", 
          "esi-skills.read_skillqueue.v1", 
          "esi-wallet.read_character_wallet.v1", 
          "esi-clones.read_clones.v1", 
          "esi-characters.read_loyalty.v1",
          "esi-characters.read_standings.v1",
          "esi-clones.read_implants.v1",
          ]
# Base Login server URL
login_server_base_url = "login.eveonline.com"
# Response type, can be either token or code, we are using code(More information can be found here - https://oauthlib.readthedocs.io/en/latest/oauth2/grants/implicit.html)
response_type = 'code'
# State for CSRF
state = 'thisshouldberandomeachrequest'
# Redirect URL, must be the same as the redirect on your CCP developer application
redirect_uri="http://localhost:65432/callback/"
# Client ID from CCP developer application goes here
client_id="22ea0b7cf4624a569c1eef3602324291"
# Secret Key from CCP developer application goes here
secret_key="9ub1OG0dtSJEzksDcIJC95xhGU4VIkMQbcmdTtQh"
# Combining Client ID and Secret Key to base64 encoded header
secret_id = client_id + ":" + secret_key
auth_b64 = "%s:%s" % (client_id, secret_key)
auth_b64 = base64.b64encode(auth_b64.encode('utf-8'))
basic = auth_b64.decode('utf-8')
# If you have it place the refresh token here, it will save having to log on everytime. Else leave blank and follow the link to login
refresh_token = ''

In [91]:
# Either refresh with the token provided or create a login URL for getting a new one.
if refresh_token: 
    refresh = requests.post('https://login.eveonline.com/v2/oauth/token', headers={'Authorization':'Basic ' + basic, "Content-Type":"application/x-www-form-urlencoded"}, data={"grant_type":"refresh_token","refresh_token":refresh_token})
    print(refresh)
    tokens = refresh.text
else:
    login_url = f"https://{login_server_base_url}/v2/oauth/authorize?response_type={response_type}&redirect_uri={redirect_uri}&client_id={client_id}&scope=publicData+esi-location.read_location.v1+esi-location.read_ship_type.v1+esi-skills.read_skills.v1+esi-skills.read_skillqueue.v1+esi-wallet.read_character_wallet.v1+esi-clones.read_clones.v1+esi-characters.read_loyalty.v1+esi-characters.read_standings.v1+esi-clones.read_implants.v1+&state={state}"
    print("Please follow this link to login. copy the code from the url after login and paste back here.")
    print(login_url)
    code = input()
    tokens = requests.post(f'https://{login_server_base_url}/v2/oauth/token', headers={'Authorization':f'Basic {basic}', "Content-Type":"application/x-www-form-urlencoded"}, data={"grant_type":"authorization_code","code":code})
    tokens = tokens.text
print(tokens)

<Response [200]>


In [92]:
tokens = json.loads(tokens)
tokens

{'access_token': 'eyJhbGciOiJSUzI1NiIsImtpZCI6IkpXVC1TaWduYXR1cmUtS2V5IiwidHlwIjoiSldUIn0.eyJzY3AiOlsicHVibGljRGF0YSIsImVzaS1sb2NhdGlvbi5yZWFkX2xvY2F0aW9uLnYxIiwiZXNpLWxvY2F0aW9uLnJlYWRfc2hpcF90eXBlLnYxIiwiZXNpLXNraWxscy5yZWFkX3NraWxscy52MSIsImVzaS1za2lsbHMucmVhZF9za2lsbHF1ZXVlLnYxIiwiZXNpLXdhbGxldC5yZWFkX2NoYXJhY3Rlcl93YWxsZXQudjEiLCJlc2ktY2xvbmVzLnJlYWRfY2xvbmVzLnYxIiwiZXNpLWNoYXJhY3RlcnMucmVhZF9sb3lhbHR5LnYxIiwiZXNpLWNoYXJhY3RlcnMucmVhZF9zdGFuZGluZ3MudjEiLCJlc2ktY2xvbmVzLnJlYWRfaW1wbGFudHMudjEiXSwianRpIjoiMmE4Y2M4MmQtNmQ3OC00MDcxLTlhZTgtNTZhMGJjYjg5OGU4Iiwia2lkIjoiSldULVNpZ25hdHVyZS1LZXkiLCJzdWIiOiJDSEFSQUNURVI6RVZFOjkyNDk3MTAyIiwiYXpwIjoiMjJlYTBiN2NmNDYyNGE1NjljMWVlZjM2MDIzMjQyOTEiLCJ0ZW5hbnQiOiJ0cmFucXVpbGl0eSIsInRpZXIiOiJsaXZlIiwicmVnaW9uIjoid29ybGQiLCJuYW1lIjoiQ29yJ0VsIERhaGtlbiIsIm93bmVyIjoiTmtGaUE3akQwWWV6NW44bEN2cytYR2M3dTBJPSIsImV4cCI6MTYzOTY4MTYzOCwiaWF0IjoxNjM5NjgwNDM4LCJpc3MiOiJsb2dpbi5ldmVvbmxpbmUuY29tIn0.kTYH8gs8TN8LuR8j_z1pxnbyRj77bYlObj0Cy6xuREXYLL5bSWvctORIP03o19H32X

In [93]:
verify = requests.get('https://login.eveonline.com/oauth/verify', headers={'Authorization': 'Bearer ' + tokens['access_token']})
verify = verify.text
info = json.loads(verify)
info

{'CharacterID': 92497102,
 'CharacterName': "Cor'El Dahken",
 'CharacterOwnerHash': 'NkFiA7jD0Yez5n8lCvs+XGc7u0I=',
 'ExpiresOn': '2021-12-16T19:07:18',
 'IntellectualProperty': 'EVE',
 'Scopes': 'publicData esi-location.read_location.v1 esi-location.read_ship_type.v1 esi-skills.read_skills.v1 esi-skills.read_skillqueue.v1 esi-wallet.read_character_wallet.v1 esi-clones.read_clones.v1 esi-characters.read_loyalty.v1 esi-characters.read_standings.v1 esi-clones.read_implants.v1',
 'TokenType': 'Character'}

Display Character location.


In [25]:

access_toke = tokens["access_token"]
#/characters/{character_id}/location/
CHARACTER_LOCATION_URL = f"https://esi.evetech.net/latest/characters/{info['CharacterID']}/location/"

req  = requests.get(
            CHARACTER_LOCATION_URL,
            headers={"Authorization": "Bearer " + access_toke},
        )
location = req.json()

#/universe/systems/
SOLAR_SYSTEM_URL = f"https://esi.evetech.net/latest/universe/systems/{location['solar_system_id']}/"
req  = requests.get(
            SOLAR_SYSTEM_URL,
            headers={"Authorization": "Bearer " + access_toke},
        )
solar_system = req.json()
print(f"System: {solar_system['name']}")

#/universe/stations/{station_id}/
STATION_URL = f"https://esi.evetech.net/latest/universe/stations/{location['station_id']}/"
req  = requests.get(
            STATION_URL,
            headers={"Authorization": "Bearer " + access_toke},
        )
station = req.json()
print(f"Station: {station['name']}")

System: Jita
Station: Jita IV - Moon 4 - Caldari Navy Assembly Plant


Display character ship.


In [30]:
access_toke = tokens["access_token"]
#/characters/{character_id}/ship/

CHARACTER_SHIP_URL = f"https://esi.evetech.net/latest/characters/{info['CharacterID']}/ship/"

req  = requests.get(
            CHARACTER_SHIP_URL,
            headers={"Authorization": "Bearer " + access_toke},
        )
ship = req.json()
print(f"Ship Name: {ship['ship_name']}")
#/universe/types/{type_id}/
UNIVERSE_TYPES_URL = f"https://esi.evetech.net/latest/universe/types/{ship['ship_type_id']}/"
req  = requests.get(
            UNIVERSE_TYPES_URL,
            headers={"Authorization": "Bearer " + access_toke},
        )
ship_type = req.json()
print(f"Ship Type: {ship_type['name']}")


Ship Name: Cor'El Dahken's Caldari Shuttle
Ship Type: Caldari Shuttle


Display character skills


In [ ]:
access_toke = tokens["access_token"]
#/characters/{character_id}/skills/

CHARACTER_SKILLS_URL = f"https://esi.evetech.net/latest/characters/{info['CharacterID']}/skills/"

req  = requests.get(
            CHARACTER_SKILLS_URL,
            headers={"Authorization": "Bearer " + access_toke},
        )
skills = req.json()
print(skills['skills'])

for skill in range(len(skills['skills'])):
    UNIVERSE_TYPES_URL = f"https://esi.evetech.net/latest/universe/types/{skills['skills'][skill]['skill_id']}/"
    req  = requests.get(
            UNIVERSE_TYPES_URL,
            headers={"Authorization": "Bearer " + access_toke},
        )
    skill_type = req.json()
    print(f"Skill: {skill_type['name']} - {skills['skills'][skill]['active_skill_level']}/{skills['skills'][skill]['trained_skill_level']}")


Display Character skillqueue

In [75]:
access_toke = tokens["access_token"]
#/characters/{character_id}/skillqueue/
CHARACTER_SKILLQ_URL = f"https://esi.evetech.net/latest/characters/{info['CharacterID']}/skillqueue/"

req  = requests.get(
            CHARACTER_SKILLQ_URL,
            headers={"Authorization": "Bearer " + access_toke},
        )
skillq = req.json()
print("Skill Queue")
for skill in range(len(skillq)):
    UNIVERSE_TYPES_URL = f"https://esi.evetech.net/latest/universe/types/{skillq[skill]['skill_id']}/"
    req  = requests.get(
            UNIVERSE_TYPES_URL,
            headers={"Authorization": "Bearer " + access_toke},
        )
    skillq_type = req.json()
    
    print(f"{skillq_type['name']} Level {skillq[skill]['finished_level']}")

Skill Queue
Scrapmetal Processing Level 5
Uncommon Moon Ore Processing Level 1
Uncommon Moon Ore Processing Level 2
Uncommon Moon Ore Processing Level 3
Uncommon Moon Ore Processing Level 4


Display characters wallet balance.

In [78]:
#/characters/{character_id}/wallet/
CHARACTER_WALLET_URL = f"https://esi.evetech.net/latest/characters/{info['CharacterID']}/wallet/"

req  = requests.get(
            CHARACTER_WALLET_URL,
            headers={"Authorization": "Bearer " + access_toke},
        )
wallet = req.json()
currency = "${:,.2f}". format(wallet)
print(f"Wallet Balance: {currency}")

Wallet Balance: $6,924,426,670.98


Display characters clones.

In [79]:
access_toke = tokens["access_token"]
#/characters/{character_id}/clones/
CHARACTER_CLONE_URL = f"https://esi.evetech.net/latest/characters/{info['CharacterID']}/clones/"

req  = requests.get(
            CHARACTER_CLONE_URL,
            headers={"Authorization": "Bearer " + access_toke},
        )
clones = req.json()
clones

{'home_location': {'location_id': 60003760, 'location_type': 'station'},
 'jump_clones': [{'implants': [],
   'jump_clone_id': 71772481,
   'location_id': 60003886,
   'location_type': 'station'}],
 'last_clone_jump_date': '2021-05-19T20:21:22Z',
 'last_station_change_date': '2018-03-19T20:10:13Z'}

In [88]:
STATION_URL = f"https://esi.evetech.net/latest/universe/stations/{clones['home_location']['location_id']}/"
req  = requests.get(
            STATION_URL,
            headers={"Authorization": "Bearer " + access_toke},
        )
home_station = req.json()
print(f"Character Home Location: {home_station['name']}")
for clone in range(len(clones['jump_clones'])):
    STATION_URL = f"https://esi.evetech.net/latest/universe/stations/{clones['jump_clones'][clone]['location_id']}/"
    req  = requests.get(
            STATION_URL,
            headers={"Authorization": "Bearer " + access_toke},
        )
    clone_station = req.json()
    print(f"Jump Clone Location: {clone_station['name']}")
    print(f"Jump Clone Implants: {clones['jump_clones'][clone]['implants']}")

Character Home Location: Jita IV - Moon 4 - Caldari Navy Assembly Plant
Jump Clone Location: Umokka X - Moon 4 - Caldari Navy Testing Facilities
Jump Clone Implants: []


Display Character Standings.

In [95]:
access_toke = tokens["access_token"]
CHARACTER_STANDINGS_URL = f"https://esi.evetech.net/latest/characters/{info['CharacterID']}/standings/"
#/characters/{character_id}/standings/
req  = requests.get(
            CHARACTER_STANDINGS_URL,
            headers={"Authorization": "Bearer " + access_toke},
        )
standings = req.json()
for entry in standings:
    if entry['from_type'] == 'npc_corp':
        CORP_INFO_URL = f"https://esi.evetech.net/latest/corporations/{entry['from_id']}/"
#/characters/{character_id}/standings/
        req  = requests.get(
            CORP_INFO_URL,
            headers={"Authorization": "Bearer " + access_toke},
            )
        corpinfo = req.json()
        print(f"Corporation: {corpinfo['name']} Standings: {entry['standing']}")

[{'from_id': 500001, 'from_type': 'faction', 'standing': 2.747732427}, {'from_id': 500002, 'from_type': 'faction', 'standing': -0.25597551700000004}, {'from_id': 500003, 'from_type': 'faction', 'standing': 1.332182032}, {'from_id': 500004, 'from_type': 'faction', 'standing': -1.867692783}, {'from_id': 500005, 'from_type': 'faction', 'standing': 0.504680789}, {'from_id': 500007, 'from_type': 'faction', 'standing': 1.910837888}, {'from_id': 500008, 'from_type': 'faction', 'standing': 1.955050326}, {'from_id': 500009, 'from_type': 'faction', 'standing': -0.33371736300000004}, {'from_id': 500010, 'from_type': 'faction', 'standing': -2.167218664}, {'from_id': 500011, 'from_type': 'faction', 'standing': -0.38860464100000003}, {'from_id': 500012, 'from_type': 'faction', 'standing': -3.578001946}, {'from_id': 500013, 'from_type': 'faction', 'standing': 0.935635819}, {'from_id': 500014, 'from_type': 'faction', 'standing': -0.6315487240000001}, {'from_id': 500015, 'from_type': 'faction', 'standi

Display Characters Loyalty Points.

In [96]:
token = tokens['access_token']
#/characters/{character_id}/loyalty/points/
LOYALTY_POINT_URL = f"https://esi.evetech.net/latest/characters/{info['CharacterID']}/loyalty/points/"
req  = requests.get(
            LOYALTY_POINT_URL,
            headers={"Authorization": f"Bearer {token}"},
            )
loyalty_points = req.json()
for entry in loyalty_points:
    CORP_INFO_URL = f"https://esi.evetech.net/latest/corporations/{entry['corporation_id']}/"
#/characters/{character_id}/standings/
    req  = requests.get(
        CORP_INFO_URL,
        headers={"Authorization": "Bearer " + access_toke},
        )
    corpinfo = req.json()
    print(f"Corporation: {corpinfo['name']} Loyalty Points: {entry['loyalty_points']}")

Corporation: Caldari Navy Loyalty Points: 193875
Corporation: Republic Security Services Loyalty Points: 67
Corporation: Emperor Family Loyalty Points: 26752
Corporation: Sisters of EVE Loyalty Points: 93
Corporation: Republic University Loyalty Points: 12


Display Characters Implants.

In [101]:
#/characters/{character_id}/implants/
IMPLANTS_URL = f"https://esi.evetech.net/latest/characters/{info['CharacterID']}/implants/"
req  = requests.get(
            IMPLANTS_URL,
            headers={"Authorization": f"Bearer {token}"},
            )
implants = req.json()
print("Character Implants:")
for implant in implants:

    UNIVERSE_TYPES_URL = f"https://esi.evetech.net/latest/universe/types/{implant}/"
    req  = requests.get(
            UNIVERSE_TYPES_URL,
            headers={"Authorization": "Bearer " + access_toke},
        )
    implant_type = req.json()
    print(implant_type['name'])

Character Implants:
Zainou 'Beancounter' Reprocessing RX-804
Ocular Filter - Improved
Memory Augmentation - Improved
Neural Boost - Improved
Cybernetic Subprocessor - Improved
